In [1]:
import tensorflow as tf
import pandas as pd
import mediapipe as mp
import cv2
import numpy as np
from yesc.utils.video import open_video

In [2]:
model = tf.keras.models.load_model('landmark-classifier')
model

In [3]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

In [4]:
def parse_landmark(hand_landmark):
    landmark = tuple((lm.x, lm.y, lm.z) for lm in hand_landmark.landmark)
    return np.array(landmark)

In [6]:
WIN_NAME = 'Camera'
hand_landmark = None

THRES = .7

with open_video(0) as video, mp_hands.Hands(static_image_mode=True,
                                            max_num_hands=2,
                                            min_detection_confidence=0.7) as hands:
    for frame in video:
        frame = cv2.flip(frame.copy(), 1)
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = hands.process(image)
        if not results.multi_hand_landmarks:
            cv2.imshow(WIN_NAME, frame)
            continue
        annotated_image = frame

        predictions = []
        for i, hand_landmarks in enumerate(results.multi_hand_landmarks):
            hand_landmark = hand_landmarks
            landmark = parse_landmark(hand_landmark)
            pred = model.predict(landmark.reshape((1, -1))).ravel()[0]

            mp_drawing.draw_landmarks(
                annotated_image,
                hand_landmarks,
                mp_hands.HAND_CONNECTIONS,
                mp_drawing_styles.get_default_hand_landmarks_style(),
                mp_drawing_styles.get_default_hand_connections_style())

            is_pos = pred > THRES

            x, y = 20, frame.shape[0] - 20 - 30 * i
            color = (0,255,0) if is_pos else (0,0,255)
            text = "Yeay" if is_pos else "Nay"
            text = f'{text} conf: {pred*100:.2f}%'
            cv2.putText(annotated_image, text,
                        (x, y),
                        fontFace=cv2.FONT_HERSHEY_SIMPLEX,
                        fontScale=.8,
                        color=color,
                        thickness=2)

        cv2.imshow(WIN_NAME, annotated_image)

cv2.destroyAllWindows()
